In [135]:
import os
import sys
from IPython import get_ipython
from pathlib import Path
import json
import pandas as pd

In [136]:
def get_script_directory():
    """
    Returns the ACTUAL directory containing the notebook/script.
    Works in:
    - VS Code Jupyter notebooks
    - Regular Jupyter Notebook/Lab
    - Standalone Python scripts
    """
    # If running in Jupyter
    if 'ipykernel' in sys.modules:
        try:
            # 1. First try VS Code's special attribute
            shell = get_ipython()
            if hasattr(shell, '__vsc_ipynb_file__'):
                return str(Path(shell.__vsc_ipynb_file__).parent)
            
            # 2. Try Jupyter notebook path (modern Jupyter)
            from notebook.notebookapp import list_running_servers
            servers = list_running_servers()
            if servers:
                import requests
                from urllib.parse import urljoin
                kernel_id = Path(get_ipython().config['IPKernelApp']['connection_file']).stem.replace('kernel-', '')
                for server in servers:
                    sessions = requests.get(urljoin(server['url'], 'api/sessions'), params={'token': server.get('token', '')}).json()
                    for session in sessions:
                        if session['kernel']['id'] == kernel_id:
                            return str(Path(server['notebook_dir']) / Path(session['notebook']['path']).parent)
            
            # 3. Fallback to current working directory
            return str(Path.cwd())
        except:
            return str(Path.cwd())
    
    # If running as a Python script
    return str(Path(__file__).parent.resolve())

In [137]:
def load_json(json_path):
    with open(json_path, 'r') as file:
        json_contents = json.load(file)
    return json_contents

In [138]:
def construct_url_dict(gid_dict, url_template):
    url_dict = {}
    for sheet, gid in gid_dict.items():
        full_url = url_template.replace("edit?gid=gid_value#gid=gid_value", f"export?format=csv&gid={gid}")
        url_dict[sheet] = full_url
    return url_dict

In [139]:
def construct_master_url_dict(sheets_dict):
    master_url_dict={}
    for spreadsheet, spreadsheet_dict in sheets_dict.items():
        spreadsheet_gid_dict=spreadsheet_dict['sheets']
        spreadsheet_url_template=spreadsheet_dict['link_template']
        spreadsheet_url_dict=construct_url_dict(spreadsheet_gid_dict, spreadsheet_url_template)
        master_url_dict[spreadsheet]=spreadsheet_url_dict
    return master_url_dict

In [140]:
def row_first_value(row):
    first_value = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else None
    return first_value

def first_column_indecies(df):
    indecies_list = df.iloc[1:, 0]
    return indecies_list

In [141]:
def initial_content_dict_from_url(url):    
    memory_df=pd.read_csv(url)
    df=memory_df
    for col in df.select_dtypes(include=['float64']):
        if (df[col].dropna().apply(float.is_integer).all()):
            df[col] = df[col].astype('Int64')
    initial_content_dict = {
        row.iloc[0]: {
            df.columns[i]: val 
            for i, val in enumerate(row.iloc[1:])  # Skip first column
            if pd.notna(val)
        }
        for _, row in df.iterrows()
        if pd.notna(row.iloc[0])  # First column as key
    }
    return initial_content_dict

In [ ]:
# def main():
script_path=get_script_directory()
config_path=script_path + '\\config.json'
sheets_path=script_path + '\\sheets.json'

config_dict=load_json(config_path)
sheets_dict=load_json(sheets_path)

master_url_dict=construct_master_url_dict(sheets_dict)

game_content_dict={}
for game in config_dict['games']:
    game_content_dict[game]={}
    for sheet, url in master_url_dict[game].items():
        game_content_dict[game][sheet]={}
        game_content_dict[game][sheet]=initial_content_dict_from_url(url)

In [ ]:
game_content_dict
    # print(game)
    # print(master_url_dict[game])

{'Cultist Simulator': {'Expeditions': {'Cater & Hero Limited': {'Unnamed: 0': 2,
    'Secret Histories Level': 'In the Capital: A secret place in the teeming capital city where I live.',
    'Aspects (tags)': "It's been abandoned since the explosion, then both the owners died. The machines stand silent; the building is a collapsing shell. Mr Cater was known to display peculiar trinkents in his office. Perhaps some yet remain.",
    'Card Text': 'The remaining factory walls could topple at any moment. Blackened timbers creak in pain. We must equip ourselves properly and move with dreadful care.',
    'Successful': "Cater's Office: Two floorboards lift away to reveal a hollow beneath Cater's desk! The rats have made free with the biscuits he stores there against the peckishness which haunts the enterpreneur, but they could not penetrate this iron lock-box. A moment with a crowbar will open it.",
    'End text': 'Treacherous Ground (Forge)',
    'Obstacle 3': 'Bronze Spintria',
    'Loot 

In [143]:
boh_memory_url=master_url_dict['Book of Hours']['Memories']
initial_content_dict_from_url(boh_memory_url)

{'A Stolen Secret': {'Heart': 1, 'Lantern': 2},
 'Ascendant Harmony': {'Knock': 2,
  'Scale': 4,
  'Winter': 'Birdsong',
  'Evolution': 'Sound',
  'Special': 'Y',
  'Persistent': 'Y'},
 'Beguiling Melody': {'Forge': 2, 'Scale': 2, 'Evolution': 'Sound'},
 'Bittersweet Certainty': {'Knock': 1, 'Sky': 2, 'Persistent': 'Y'},
 'Cheerful Ditty': {'Grail': 2,
  'Scale': 1,
  'Evolution': 'Sound',
  'Persistent': 'Y'},
 'Confounding Parable': {'Lantern': 2,
  'Nectar': 2,
  'Scale': 2,
  'Persistent': 'Y'},
 'Contradiction': {'Memory': 2, 'Lantern': 1},
 'Curious Hunch': {'Grail': 3,
  'Heart': 4,
  'Knock': 3,
  'Moon': 3,
  'Winter': 'Nyctodromy',
  'Evolution': 'Omen',
  'Special': 'Y',
  'Persistent': 'Y'},
 'Didumos': {'Heart': 3,
  'Knock': 3,
  'Scale': 6,
  'Sky': 3,
  'Winter': 'Horomachistry/Hushery',
  'Special': 'Y',
  'Persistent': 'Y'},
 'Earth-Sign': {'Moth': 2,
  'Rose': '2',
  'Winter': 'Skolekosophy',
  'Evolution': 'Omen',
  'Special': 'Y'},
 'Earthquake Name': {'Edge': 3,
 

In [144]:
# # Create output directory relative to script location
# script_dir = Path(get_script_directory())
# output_dir = script_dir.parent / "Obsidian/markdown_files"
# output_dir.mkdir(exist_ok=True)

# def row_value_pairs(row):
#     """Convert a pandas row to Obsidian-friendly markdown format"""
#     content = []
    
#     # Get the first column's value (regardless of other columns)
#     first_col_name = row.index[0]  # Name of the first column
    
#     # Add other columns as key-value pairs (skip the first column)
#     # content.append(f"- **Type**: {type}")
#     for col, val in row.items():
#         if col == first_col_name:  # Skip the first column (already used as heading)
#             content.append(f"- **Type**: {col}")
#         if pd.notna(val):
#             clean_val = str(val).strip().replace('\r\n', '\n').replace('\n', '<br>')
#             content.append(f"- **{col}**: {clean_val}")
    
#     return "\n".join(content)

# def md_files_from_df(df):
#     # Write each row to a markdown file
#     for index, row in df.iterrows():
#         # Create safe filename (remove special chars)
#         safe_filename = row_first_value(row).replace(':', ' -') + '.md'
#         filepath = output_dir / safe_filename
#         # print(f'filepath: {filepath}')
        
#         try:
#             with open(filepath, 'w', encoding='utf-8') as f:
#                 f.write(row_value_pairs(row))
#             print(f"✓ Created: {filepath.relative_to(output_dir)}")
#         except Exception as e:
#             print(f"✗ Error writing {filepath.name}: {str(e)}")